In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt

In [ ]:
def create_model(hp):
    nn = tf.keras.models.Sequential()
        # Define Model
    activation = hp.Choice("activation", ["relu", "tanh"])
        # Allow model to choose activation mode
    nn.add( # add the first layer
        tf.keras.layers.Dense(
            units=hp.Int('first_units', min_value=10, max_value=100),
            activation=activation,
            input_dim=44
            )
        )
    # I am going to let keras decide how many layers the model should have
    for i in range(hp.Int("num_layers", 1, 5)):
        nn.add(
            tf.keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=10, max_value=100),
                activation=activation
            )
        )
    
    nn.add( # add the output layer
        tf.keras.layers.Dense(
            units=1,
            activation='sigmoid'
        )
    )
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
        # compile the model
    return nn # output the model

In [4]:
tuner2 = kt.Hyperband(
    create_model, objective="val_accuracy", max_epochs=100, hyperband_iterations=2
)

Reloading Tuner from .\untitled_project\tuner0.json


In [16]:
df = pd.read_csv('processed_dat.csv')

In [17]:
y = df['IS_SUCCESSFUL'] # target
X = df.drop('IS_SUCCESSFUL', axis=1) # features
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1) # split it up

In [18]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
tuner2.search(
    X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test)
)

Trial 508 Complete [00h 01m 35s]
val_accuracy: 0.7306122183799744

Best val_accuracy So Far: 0.7341107726097107
Total elapsed time: 02h 03m 40s


In [5]:
top_hyper = tuner2.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 40, 'num_layers': 2, 'units_0': 11, 'units_1': 13, 'units_2': 39, 'units_3': 43, 'units_4': 18, 'tuner/epochs': 34, 'tuner/initial_epoch': 12, 'tuner/bracket': 3, 'tuner/round': 2, 'tuner/trial_id': '0194'}
{'activation': 'relu', 'first_units': 100, 'num_layers': 2, 'units_0': 52, 'units_1': 96, 'units_2': 81, 'units_3': 43, 'units_4': 10, 'tuner/epochs': 4, 'tuner/initial_epoch': 0, 'tuner/bracket': 3, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 60, 'num_layers': 2, 'units_0': 93, 'units_1': 86, 'units_2': 90, 'units_3': 61, 'units_4': 62, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0459'}


In [10]:
top_model = tuner2.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

c:\Users\Leo\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Leo\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


268/268 - 0s - 2ms/step - accuracy: 0.7341 - loss: 0.5510
Loss: 0.5510062575340271, Accuracy: 0.7341107726097107
268/268 - 0s - 1ms/step - accuracy: 0.7336 - loss: 0.5536
Loss: 0.553550660610199, Accuracy: 0.7336443066596985
268/268 - 0s - 1ms/step - accuracy: 0.7333 - loss: 0.5501
Loss: 0.5500675439834595, Accuracy: 0.7332944869995117


In [12]:
def create_model2(hp):
    nn = tf.keras.models.Sequential()
        # Define Model
    activation = hp.Choice("activation", ["relu", "tanh"])
        # Allow model to choose activation mode
    nn.add( # add the first layer
        tf.keras.layers.Dense(
            units=hp.Int('first_units', min_value=10, max_value=100),
            activation=activation,
            input_dim=44
            )
        )
    # I am going to let keras decide how many layers the model should have
    for i in range(hp.Int("num_layers", 2, 7)):
        nn.add(
            tf.keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=10, max_value=100),
                activation=activation
            )
        )
    
    nn.add( # add the output layer
        tf.keras.layers.Dense(
            units=1,
            activation='sigmoid'
        )
    )
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
        # compile the model
    return nn # output the model

In [13]:
tuner3 = kt.Hyperband(
    create_model2, objective="val_accuracy", max_epochs=100, hyperband_iterations=2, overwrite=True
)

c:\Users\Leo\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
tuner3.search(
    X_train_scaled, y_train, epochs=10,validation_data=(X_test_scaled, y_test)
)

Trial 500 Complete [00h 01m 48s]
val_accuracy: 0.7315452098846436

Best val_accuracy So Far: 0.7343440055847168
Total elapsed time: 02h 19m 47s


In [21]:
top_hyper = tuner3.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 34, 'num_layers': 3, 'units_0': 78, 'units_1': 21, 'units_2': 15, 'units_3': 70, 'units_4': 87, 'units_5': 99, 'units_6': 26, 'tuner/epochs': 34, 'tuner/initial_epoch': 12, 'tuner/bracket': 4, 'tuner/round': 3, 'tuner/trial_id': '0383'}
{'activation': 'relu', 'first_units': 34, 'num_layers': 3, 'units_0': 78, 'units_1': 21, 'units_2': 15, 'units_3': 70, 'units_4': 87, 'units_5': 99, 'units_6': 26, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 4, 'tuner/round': 4, 'tuner/trial_id': '0388'}
{'activation': 'tanh', 'first_units': 68, 'num_layers': 5, 'units_0': 21, 'units_1': 32, 'units_2': 84, 'units_3': 16, 'units_4': 19, 'units_5': 45, 'units_6': 71, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0196'}


In [22]:
top_model = tuner3.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

c:\Users\Leo\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Leo\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\Leo\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


268/268 - 0s - 1ms/step - accuracy: 0.7343 - loss: 0.5504
Loss: 0.5504232048988342, Accuracy: 0.7343440055847168
268/268 - 0s - 1ms/step - accuracy: 0.7332 - loss: 0.5534
Loss: 0.5534474849700928, Accuracy: 0.7331778407096863
268/268 - 0s - 1ms/step - accuracy: 0.7331 - loss: 0.5523
Loss: 0.5523414015769958, Accuracy: 0.7330612540245056


## Best Model Acheived:

[`untitled_project/trial_0383/build_config.json`](untitled_project/trial_0383/build_config.json) <-- Input Shape
[`untitled_project/trial_0383/trial.json`](untitled_project/trial_0383/trial.json) <-- Parameters

"values": 

        {"activation": "relu",
        "first_units": 34,
        "num_layers": 3,
        "units_0": 78,
        "units_1": 21,
        "units_2": 15,
        "tuner/epochs": 12,
        "tuner/initial_epoch": 4,
        "tuner/bracket": 4,
        "tuner/round": 2,
        "tuner/trial_id": "0375"}